## Deliverable 2. Create a Customer Travel Destinations Map.

In [10]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [12]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,82.40,74,75,13.80,light rain
1,1,Rikitea,PF,-23.1203,-134.9692,77.81,73,96,12.97,overcast clouds
2,2,Lebu,CL,-37.6167,-73.6500,60.71,87,24,3.65,few clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,60.80,59,0,4.61,clear sky
4,4,Lavrentiya,RU,65.5833,-171.0000,-8.27,85,0,7.85,clear sky


In [13]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 68
What is the maximum temperature you would like for your trip? 85


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,82.40,74,75,13.80,light rain
1,1,Rikitea,PF,-23.1203,-134.9692,77.81,73,96,12.97,overcast clouds
6,6,Dingle,PH,10.9995,122.6711,76.06,91,100,9.22,overcast clouds
8,8,Mataura,NZ,-46.1927,168.8643,70.38,46,99,3.13,overcast clouds
11,11,East London,ZA,-33.0153,27.9116,71.71,84,94,10.65,light rain
12,12,Balikpapan,ID,-1.2675,116.8289,78.80,94,40,3.44,scattered clouds
16,16,Punta Arenas,CL,-53.1500,-70.9167,71.60,24,0,13.80,clear sky
19,19,Samarai,PG,-10.6167,150.6667,84.65,69,98,4.12,overcast clouds
20,20,Sao Joao Do Piaui,BR,-8.3581,-42.2467,80.33,60,31,3.11,scattered clouds
27,27,Trincomalee,LK,8.5711,81.2335,78.40,84,24,19.24,few clouds


In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                234
City                   234
Country                232
Lat                    234
Lng                    234
Max Temp               234
Humidity               234
Cloudiness             234
Wind Speed             234
Current Description    234
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any')
clean_df.count()

City_ID                232
City                   232
Country                232
Lat                    232
Lng                    232
Max Temp               232
Humidity               232
Cloudiness             232
Wind Speed             232
Current Description    232
dtype: int64

In [31]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,82.40,light rain,-0.7393,-90.3518,
1,Rikitea,PF,77.81,overcast clouds,-23.1203,-134.9692,
6,Dingle,PH,76.06,overcast clouds,10.9995,122.6711,
8,Mataura,NZ,70.38,overcast clouds,-46.1927,168.8643,
11,East London,ZA,71.71,light rain,-33.0153,27.9116,
12,Balikpapan,ID,78.80,scattered clouds,-1.2675,116.8289,
16,Punta Arenas,CL,71.60,clear sky,-53.1500,-70.9167,
19,Samarai,PG,84.65,overcast clouds,-10.6167,150.6667,
20,Sao Joao Do Piaui,BR,80.33,scattered clouds,-8.3581,-42.2467,
27,Trincomalee,LK,78.40,few clouds,8.5711,81.2335,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [39]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.drop(hotel_df[hotel_df['Hotel Name'] == ""].index, inplace = True) 
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,82.40,light rain,-0.7393,-90.3518,Castillo Galapagos
1,Rikitea,PF,77.81,overcast clouds,-23.1203,-134.9692,Pension Maro'i
6,Dingle,PH,76.06,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
8,Mataura,NZ,70.38,overcast clouds,-46.1927,168.8643,Ellie's Villa
11,East London,ZA,71.71,light rain,-33.0153,27.9116,Tu Casa
12,Balikpapan,ID,78.80,scattered clouds,-1.2675,116.8289,Grand Jatra Hotel Balikpapan
16,Punta Arenas,CL,71.60,clear sky,-53.1500,-70.9167,Hotel Dreams Del Estrecho
19,Samarai,PG,84.65,overcast clouds,-10.6167,150.6667,Nuli Sapi
20,Sao Joao Do Piaui,BR,80.33,scattered clouds,-8.3581,-42.2467,Nayra Palace Hotel
27,Trincomalee,LK,78.40,few clouds,8.5711,81.2335,Welcombe Hotel


In [40]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [53]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [54]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))